In [20]:
from __future__ import print_function
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# os.environ["CUDA_VISIBLE_DEVICES"] = "2" (if you want to use GPU2)

import sys
import math
import json
import pickle
import numpy as np
import tensorflow as tf



import json
sentence_list_fname = './assignment_training_data_word_segment.json'
sentence_list = json.load(open(sentence_list_fname , 'r'))

# How to load vocabulary in Python:

import pickle # (in python 3.x)
# import cPickle as pickle # (in python 2.x)

# voc_dict_fname = './voc-d.pkl'
# voc_dict = pickle.load(open(voc_dict_fname, 'rb'))
# idx2word, word2idx = voc_dict['idx2word'], voc_dict['word2idx'] 
# idx2word[index] is a word
# word2idx[word] is an index
tf.reset_default_graph()

def process(data):
    result=[]
    num =0
    index = 0
    for sentence in data:
        t_pos = -1
        t_num = len(sentence['times'])
        a_num = len(sentence['attributes'])
        v_num = len(sentence['values'])
        sentence['testset']=[]
        comma = [0]*5
        semicolon = 0
        comma_now = 0
        for j in sentence['indexes']:
            if j == 7 :
                comma[comma_now] = j
                comma_now +=1
            if comma_now > 4:
                break
        for j in sentence['indexes']:
            if j == 26 :
                semicolon = j
                break   
        for t in sentence['times']:
            t_pos += 1
            a_pos = -1
            for a in sentence['attributes']:
                a_pos += 1
                v_pos = -1
                for v in sentence['values']:
                    v_pos += 1
                    origin_triple=[t,a,v]
                    true=[0]*2
                    true[0]=1
                    for i in sentence['results']:
                        if((i[0]==origin_triple[0]) and(i[1]==origin_triple[1]) and (i[2]==origin_triple[2])):
                            true[1]=1
                            true[0]=0
                            break
#                         print(num,v_num,v_pos)
#                     w_info=[0]*(225+225+400+20)
#                     w_info[t_num*15 + t_pos] = 1
#                     w_info[225+a_num*15 + a_pos] = 1
#                     w_info[450+v_num*20 + v_pos] = 1
#                     w_info[851] = a-t
#                     w_info[852] = v-a
#                     w_info[853] = t_pos
#                     w_info[854] = a_pos
#                     w_info[855] = v_pos
#                     w_info[856] = t_num
#                     w_info[857] = a_num
#                     w_info[858] = v_num
#                     w_info[859] = t
#                     w_info[860] = a
#                     w_info[861] = v
#                     w_info[862] = comma
#                     w_info[863] = semicolon
#                     w_info[675] = a-t
#                     w_info[676] = v-a
#                     w_info[677] = t_pos
#                     w_info[678] = a_pos
#                     w_info[679] = v_pos
#                     w_info[680] = t_num
#                     w_info[681] = a_num
#                     w_info[682] = v_num
#                     w_info[683] = t
#                     w_info[684] = a
#                     w_info[685] = v
                    w_info=[0]*20
                    w_info[0] = t_pos
                    w_info[1] = a_pos
                    w_info[2] = v_pos
                    w_info[3] = t_num
                    w_info[4] = a_num
                    w_info[5] = v_num
                    w_info[6] = t
                    w_info[7] = a
                    w_info[8] = v
                    w_info[9] = comma[0]
                    w_info[10] = comma[1]
                    w_info[11] = comma[2]
                    w_info[12] = comma[3]
                    w_info[13] = comma[4]
                    w_info[14] = semicolon
                    w_info[15] = t-a
                    w_info[16] = a-v
                    w_info[17] = comma[0] - t
                    w_info[18] = comma[0] - a
                    w_info[19] = comma[1] - v
                    sentence['testset'].append([true, w_info,origin_triple])
                    result.append([true, w_info,origin_triple])
                    num+=1
#                     if num > 1000:
#                         index =1
#                         break
#                 if index == 1:
#                     break
#             if index == 1:
#                 break
                
    return result,num

data,num = process(sentence_list)
print(num)
print(data[0],data[1],data[2])

train_rate=0.001
train_step=10000
batch_size=2560


frame_size=20
# sequence_length=58
sequence_length= 1
hidden_num=128
n_classes=2

#定义输入,输出
x=tf.placeholder(dtype=tf.float32,shape=[None,sequence_length*frame_size],name="x")
y=tf.placeholder(dtype=tf.float32,shape=[None,n_classes],name="y")
#定义权值
weights=tf.Variable(tf.truncated_normal(shape=[hidden_num,n_classes]))
bias=tf.Variable(tf.zeros(shape=[n_classes]))

# def get_rnn_cell():
#     return tf.nn.rnn_cell.BasicRNNCell(hidden_num)

def RNN(x,weights,bias):
    x=tf.reshape(x,shape=[-1,sequence_length,frame_size])
    rnn_cell_mine=tf.nn.rnn_cell.BasicRNNCell(hidden_num)
#     rnn_cell_mine=tf.nn.rnn_cell.MultiRNNCell([rnn_cell_mine] * 3)
    init_state=tf.zeros(shape=[batch_size,rnn_cell_mine.state_size])
    output,states=tf.nn.dynamic_rnn(rnn_cell_mine,x,dtype=tf.float32)
#     print(output)
    return tf.nn.softmax(tf.matmul(output[:,-1,:],weights)+bias,1)
predy=RNN(x,weights,bias)
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predy,labels=y))
# print(predy)
# pred = predy[0]
# print(y)
# y_ = y[0]
# cost_=np.square(predy - y)
# print(cost_)
# cost=tf.reduce_mean(cost_)
# # cost=tf.losses.mean_squared_error(cost_)
train=tf.train.AdamOptimizer(train_rate).minimize(cost)

correct_pred=tf.equal(tf.argmax(predy,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.to_float(correct_pred))


def next_batch(num_in,total,current_pos):
    result_x = []
    result_y = []
    for i in range(num_in):
        sentence_current = data[current_pos] 
        result_x.append(sentence_current[1])
        result_y.append(sentence_current[0])
        current_pos+=1
        if (current_pos ==  total -1):
            current_pos = 0
    result_y = np.reshape(result_y,[-1,2])
#     print(current_pos)
    return result_x,result_y,current_pos

def test_next_batch(num):
    result_x = []
    result_y = []
    for i in range(num):
        sentence_current = data[i] 
        result_x.append(sentence_current[1])
        result_y.append(sentence_current[0])
    result_y = np.reshape(result_y,[-1,2])
    return result_x,result_y



sess=tf.Session()
sess.run(tf.initialize_all_variables())
step=1
test_x,test_y=test_next_batch(num)


batch_num = num
i=0
epochs =30
current_pos = 0
for epoch in range(epochs):
    # for i in range(batch_num):
    # sentence_current = data[i]
    # batch_x, batch_y = np.reshape(sentence_current[1],[-1,900]), np.reshape(sentence_current[0],[-1,1])
    for step in range(50):
        batch_x, batch_y,current_pos = next_batch(batch_size,batch_num,current_pos)
    #       batch_x=tf.reshape(batch_x,shape=[batch_size,sequence_length,frame_size])
        _loss,__=sess.run([cost,train],feed_dict={x:batch_x,y:batch_y})
            
        step+=1
        if step  %5 ==0:
            acc,loss=sess.run([accuracy,cost],feed_dict={x:batch_x,y:batch_y})
            print("epoch:",epoch,"step:",step,"acc:",acc,"loss:",loss)
# #     _loss,__=sess.run([cost,train],feed_dict={x:test_x,y:test_y})
# #     acc,loss=sess.run([accuracy,cost],feed_dict={x:test_x,y:test_y})
#     print("!epoch:",epoch,"acc:",acc,"loss:",loss)
            



126208
[[0, 1], [0, 0, 0, 3, 1, 3, 0, 8, 11, 7, 7, 7, 7, 7, 0, -8, -3, 7, -1, -4], [0, 8, 11]] [[1, 0], [0, 0, 1, 3, 1, 3, 0, 8, 13, 7, 7, 7, 7, 7, 0, -8, -5, 7, -1, -6], [0, 8, 13]] [[1, 0], [0, 0, 2, 3, 1, 3, 0, 8, 15, 7, 7, 7, 7, 7, 0, -8, -7, 7, -1, -8], [0, 8, 15]]
epoch: 0 step: 5 acc: 0.8871094 loss: 0.42398387
epoch: 0 step: 10 acc: 0.878125 loss: 0.4351472
epoch: 0 step: 15 acc: 0.8625 loss: 0.45076266
epoch: 0 step: 20 acc: 0.9238281 loss: 0.38943443
epoch: 0 step: 25 acc: 0.9390625 loss: 0.37434834
epoch: 0 step: 30 acc: 0.95585936 loss: 0.35740352
epoch: 0 step: 35 acc: 0.9195312 loss: 0.39373127
epoch: 0 step: 40 acc: 0.9652344 loss: 0.34802812
epoch: 0 step: 45 acc: 0.9441406 loss: 0.36912194
epoch: 0 step: 50 acc: 0.91328126 loss: 0.39998132
epoch: 1 step: 5 acc: 0.92226565 loss: 0.39099693
epoch: 1 step: 10 acc: 0.9453125 loss: 0.36795005
epoch: 1 step: 15 acc: 0.9351562 loss: 0.37810627
epoch: 1 step: 20 acc: 0.93125 loss: 0.38201252
epoch: 1 step: 25 acc: 0.92265624 l

epoch: 15 step: 50 acc: 0.9554688 loss: 0.35779393
epoch: 16 step: 5 acc: 0.94257814 loss: 0.3706844
epoch: 16 step: 10 acc: 0.9171875 loss: 0.39607507
epoch: 16 step: 15 acc: 0.9355469 loss: 0.37771565
epoch: 16 step: 20 acc: 0.9410156 loss: 0.3722469
epoch: 16 step: 25 acc: 0.9332031 loss: 0.3800599
epoch: 16 step: 30 acc: 0.96015626 loss: 0.35310626
epoch: 16 step: 35 acc: 0.9230469 loss: 0.39021564
epoch: 16 step: 40 acc: 0.9015625 loss: 0.41170007
epoch: 16 step: 45 acc: 0.89921874 loss: 0.41404384
epoch: 16 step: 50 acc: 0.9398438 loss: 0.37341878
epoch: 17 step: 5 acc: 0.93007815 loss: 0.38318437
epoch: 17 step: 10 acc: 0.91484374 loss: 0.3984188
epoch: 17 step: 15 acc: 0.94492185 loss: 0.36834073
epoch: 17 step: 20 acc: 0.9488281 loss: 0.3644344
epoch: 17 step: 25 acc: 0.9597656 loss: 0.35349712
epoch: 17 step: 30 acc: 0.9230469 loss: 0.39021564
epoch: 17 step: 35 acc: 0.9152344 loss: 0.3980282
epoch: 17 step: 40 acc: 0.90546876 loss: 0.40779382
epoch: 17 step: 45 acc: 0.913281